In [1]:
import ipywidgets as widgets
import pandas as pd
import datetime
from time import time

from IPython.display import display, clear_output

In [2]:
### This notebook has been created with the help of ChatGPT. ###

In [3]:
pd.set_option('display.max_colwidth', None)  # For older versions of pandas, use -1 instead of None
pd.set_option('display.max_columns', None)   # Ensure all columns are shown
pd.set_option('display.max_rows', None)   

In [4]:
df = pd.read_csv('data/annotation_df_history/isaiah_dataset.csv')  # Replace with your DataFrame
df = df.fillna("")

# converting 'dir_he' and 'dir_he_dss' from float to int 

df["dir_he"] = df["dir_he"].astype(str)
df["dir_he"] = df["dir_he"].replace("1.0", "1").replace("0.0", "0")

df = df[[ 
    'verb_id',
    'lex',
    'scroll',
    'book',
    'chapter',
    'verse_num',
    'gcons_verb',
    'gcons_verse',
    'sign_info',
    'stem',
    'tense','gcons_clause',
    'subject',
    'complement',
    'cmpl_lex',
    "cmpl_translation",
    'dir_he',
    'cmpl_constr', 
    'cmpl_nt', 
    'cmpl_anim', 
    'cmpl_det', 
    'cmpl_indiv',
    'cmpl_complex',
    'motion_type',
    'preposition_1',
    'preposition_2',
    'preposition_3',
    'preposition_4',]]
#df

### Step 3: Define Columns to Annotate and Create Widgets

In [5]:
# List of columns of interest
columns = df.columns.to_list()
columns_for_info = [
    'verb_id',
    'lex',
    'scroll',
    'book',
    'chapter',
    'verse_num',
    'gcons_verb',
    'gcons_verse',
    'sign_info',
    'stem',
    'tense',
]
columns_line2 = [
    'gcons_clause',
    'subject',
    'complement',
    'cmpl_lex',
    "cmpl_translation",
    'dir_he',
    'cmpl_constr', 
    'cmpl_nt', 
    'cmpl_anim', 
    'cmpl_det', 
    'cmpl_indiv',
    'cmpl_complex',
    
]

columns_line3 = [   
    'motion_type',
    'preposition_1',
    'preposition_2',
    'preposition_3',
    'preposition_4',   
]

columns_to_annotate = columns_line2 + columns_line3

In [6]:
annotation_input = widgets.Textarea(
    value='',
    placeholder='Type your annotation here',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')  # Adjust width and height as needed
)

prev_col_button = widgets.Button(
    description='Previous Column',
    disabled=False,
    button_style='primary',
    tooltip='Go to the previous column',
    icon='arrow-left'
)

next_col_button = widgets.Button(
    description='Next Column',
    disabled=False,
    button_style='info',
    tooltip='Go to the next column',
    icon='arrow-right',
)

prev_row_button = widgets.Button(
    description='Previous Row',
    disabled=False,
    button_style='warning',
    tooltip='Go to the previous row',
    icon='arrow-left'
)

next_row_button = widgets.Button(
    description='Next Row',
    disabled=False,
    button_style='success',
    tooltip='Go to the next row',
    icon='arrow-right'
)

In [7]:
# Get session start index

def get_session_start_index():
    with open('data/annotation_df_history/annotation_tracks.txt') as annotation_file:
        lines = annotation_file.readlines()
        lines_with_index = [line for line in lines if line.startswith("Current Row Index: ")]
        line = lines_with_index[-1]
        return int(line.split()[-1])

### Step 4.1 Add function for tracking your progress

In [8]:
# Today's dataframe
df_name = "isaiah_dataset"

# Global variable to store the start index of the annotation session
session_start_index = get_session_start_index()
#print(session_start_index)

In [9]:
def save_annotation_details(df, current_index, session_start_index):
    file_path = 'data/annotation_df_history/annotation_tracks.txt'
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    current_row_index = df.index[current_index]

    # Calculate the number of rows annotated in this session
    rows_annotated_this_session = current_index - session_start_index + 1

    details = f"Date and Time: {current_datetime}\nDataFrame: {df_name}\nCurrent Row Index: {current_row_index}\nRows Annotated This Session: {rows_annotated_this_session}\n\n"

    with open(file_path, 'a') as file:
        file.write(details)

        
#def count_fully_annotated_rows(df, columns_to_annotate):
    #return df[columns_to_annotate].dropna().shape[0]


def save_dataframe(df, df_name):
    # Get the current date in YYYY-MM-DD format
    current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")

    # Construct the filename
    filename = f"data/annotation_df_history/{df_name}_{current_date}.csv"
    current_file_name = f"data/annotation_df_history/{df_name}.csv"

    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)  # Set index=False if you don't want to save the index
    df.to_csv(current_file_name, index=False)

    print(f"DataFrame saved as {filename} and {current_file_name}.")

### Step 4.2 Functions to Display Data and Widgets with Navigation and Iteration Logic

In [10]:
def display_row(row_index, col_index):
    global current_index, current_column_index
    current_index, current_column_index = row_index, col_index
    
    # Get the value from the DataFrame, convert NaN or non-string values to string
    cell_value = df.at[row_index, columns_to_annotate[col_index]]
    if pd.isna(cell_value):
        cell_value = ''  # Convert NaN to an empty string
    else:
        cell_value = str(cell_value)  # Convert non-string values to string

    clear_output(wait=True)

    # Set the value of the annotation input and display it along with other elements
    annotation_input.value = cell_value
    display(df.iloc[row_index:row_index+1][columns_for_info])
    display(df.iloc[row_index:row_index+1][columns_line2])
    display(df.iloc[row_index:row_index+1][columns_line3])
    #display(df.iloc[row_index][columns_for_info])
    #display(df.iloc[row_index][columns_to_annotate])
    display(widgets.HTML(value=f"<b>Annotate '{columns_to_annotate[col_index]}':</b>"))
    display(annotation_input)
    display(widgets.HBox([prev_col_button, next_col_button]))
    display(widgets.HBox([prev_row_button, next_row_button]))

def navigate_row(offset):
    new_index = current_index + offset
    if 0 <= new_index < len(df):
        display_row(new_index, 0)

### Step 5: Handle Annotation Submission and Row Navigation

In [11]:
def on_prev_col_clicked(b):    
    global current_column_index
    # Save the current annotation
    df.at[current_index, columns_to_annotate[current_column_index]] = annotation_input.value

    # Navigate to the previous column
    if current_column_index > 0:
        current_column_index -= 1
        display_row(current_index, current_column_index)

        
def on_next_col_clicked(b):
    global current_column_index
    # Save the current annotation
    df.at[current_index, columns_to_annotate[current_column_index]] = annotation_input.value

    # Check if the current column is the last one in the row
    if current_column_index + 1 < len(columns_to_annotate):
        # Not the last column, move to the next column
        current_column_index += 1
        display_row(current_index, current_column_index)
    else:
        # Last column, all annotations for this row are complete
        print("All annotations for this row are complete.")
        # Optional: Reset column index or navigate to the next row

        
def on_prev_row_clicked(b):
    navigate_row(-1)

def on_next_row_clicked(b):
    navigate_row(1)

In [12]:
if "on_click_defined" not in globals():
    on_click_defined = True
    prev_col_button.on_click(on_prev_col_clicked)
    next_col_button.on_click(on_next_col_clicked)

    prev_row_button.on_click(on_prev_row_clicked)
    next_row_button.on_click(on_next_row_clicked)

### Step 6: Annotate!

In [13]:
current_index = get_session_start_index()
current_column_index = 0  # Start with the first column to annotate
display_row(current_index, current_column_index)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense
186,1898844,CWB[,1Qisaa,Isaiah,10,22,JCWB,KJ> >M JHJH <MK JFR>L K XWL H JM C>R JCWB BW KLJWN XRWY CWVP YDQH,000 00 0000 0000 00000 0 000 0 00 000 0000 000 00000 0000 0000 0000 00,qal,impf


,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex
186,KJ> >M JHJH <MK JFR>L K XWL H JM C>R JCWB BW,C>R,BW,B,to him,0,prep,prps,anim,det,prsf,simple


,motion_type,preposition_1,preposition_2,preposition_3,preposition_4
186,fictive,B,,,


HTML(value="<b>Annotate 'gcons_clause':</b>")

Textarea(value='KJ> >M JHJH <MK JFR>L K XWL H JM C>R JCWB BW', layout=Layout(height='100px', width='100%'), pl…

In [ ]:
current_index

In [38]:
df.iloc[[184]]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4
184,215910,CWB[,MT,Isaiah,10,22,JCWB,KJ >M JHJH <MK JFR>L K XWL H JM C>R JCWB BW KLJWN XRWY CWVP YDQH,,qal,impf,KJ C>R JCWB BW,C>R,BW,B,to him,0,prep,prps,anim,det,prsf,simple,fictive,B,,,


### Name types:
- pers    person
- god     god
- mens    measurement unit
- gens    people
- topo    place
- dir     direction 
- ppde    demonstrative personal pronoun
- ppin    interrogative pronoun
- bopa    body part
- prps    personal pronoun
- abs     abstract nouns (freedom, judgments, etc)
- adv     adverb
- obj     object, thing
- other   other
- na      non applicable

### Animacy:
- anim
- inanim
- anim inanim (for names like Judah which can be either a people or a place or both
- unknown

### Individuation
- subs    substantive
- nmpr    proper noun
- prsf    pronominal suffix
- ppin    interrogative pronoun
- adv     adverb

### Motion type
- factive
- fictive
- factive fictive : I do not know! (interpretation needed)
- unknown: other reason

### Factive motion checklist:
- something or someone *physical* in the expression goes through a change of position from a point A or to a point B (or both)


### Step 7: Save your progress in a text file

In [42]:
#fully_annotated_rows = count_fully_annotated_rows(df, columns_to_annotate)
save_annotation_details(df, current_index, session_start_index)

In [43]:
# Example call - replace 'df' and 'DataFrame_Name' with your actual DataFrame variable and its name
save_dataframe(df, df_name)

DataFrame saved as data/annotation_df_history/isaiah_dataset_2023-12-07_17_03.csv and data/annotation_df_history/isaiah_dataset.csv.


In [44]:
current_index - session_start_index

31